In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
tickers = ["aapl", "msft", "amzn", "tsla", "googl"]
df = {}
for i in tickers:
    file_path = "/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Stocks/" + i + ".us.txt"
    dft = pd.read_csv(file_path, parse_dates=["Date"])
    dft["Ticker"] = i
    df[i] = dft
# Combine into a single DataFrame
df_combined = pd.concat(df.values())

# Set MultiIndex: Ticker (Outer), Date (Inner)
df_combined.set_index(["Ticker", "Date"], inplace=True)

# Display the first few rows
print(df_combined.head())

                      Open     High      Low    Close    Volume  OpenInt
Ticker Date                                                             
aapl   1984-09-07  0.42388  0.42902  0.41874  0.42388  23220030        0
       1984-09-10  0.42388  0.42516  0.41366  0.42134  18022532        0
       1984-09-11  0.42516  0.43668  0.42516  0.42902  42498199        0
       1984-09-12  0.42902  0.43157  0.41618  0.41618  37125801        0
       1984-09-13  0.43927  0.44052  0.43927  0.43927  57822062        0


In [3]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 26691 entries, ('aapl', Timestamp('1984-09-07 00:00:00')) to ('googl', Timestamp('2017-11-10 00:00:00'))
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Open     26691 non-null  float64
 1   High     26691 non-null  float64
 2   Low      26691 non-null  float64
 3   Close    26691 non-null  float64
 4   Volume   26691 non-null  int64  
 5   OpenInt  26691 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 1.6+ MB


In [4]:
missing_counts = df_combined.isnull().groupby(level="Ticker").sum()
print(missing_counts)


        Open  High  Low  Close  Volume  OpenInt
Ticker                                         
aapl       0     0    0      0       0        0
amzn       0     0    0      0       0        0
googl      0     0    0      0       0        0
msft       0     0    0      0       0        0
tsla       0     0    0      0       0        0


no missing values, hence skipping this step

In [5]:
df_combined.reset_index(inplace=True) 
df_combined["Date"] = pd.to_datetime(df_combined["Date"])  
df_combined.set_index(["Ticker", "Date"], inplace=True)


In [6]:
df_combined = df_combined.sort_index(level=["Ticker", "Date"])

import datetime

cutoff_date = datetime.datetime(2007, 11, 10)
df_filtered = df_combined.loc[df_combined.index.get_level_values("Date") >= cutoff_date]

print(df_filtered.head())
print(df_filtered.index.get_level_values("Date").min())

                     Open    High     Low   Close     Volume  OpenInt
Ticker Date                                                          
aapl   2007-11-12  21.130  21.479  19.291  19.691  492362604        0
       2007-11-13  20.615  21.897  19.691  21.765  484373501        0
       2007-11-14  22.733  22.739  20.970  21.274  403585172        0
       2007-11-15  21.280  21.717  20.528  21.040  414487458        0
       2007-11-16  21.193  21.388  20.405  21.309  385660112        0
2007-11-12 00:00:00


In [7]:
df_filtered = df_filtered.copy()
df_filtered["Daily Return"] = df_filtered["Close"].pct_change() * 100
df_filtered["7d MA"] = df_filtered["Close"].rolling(window=7).mean()
df_filtered["30d MA"] = df_filtered["Close"].rolling(window=30).mean()
df_filtered["Rolling Volatility (30d)"] = df_filtered["Daily Return"].rolling(window=30).std()
print(df_filtered.head())
print(df_filtered.info())  

                     Open    High     Low   Close     Volume  OpenInt  \
Ticker Date                                                             
aapl   2007-11-12  21.130  21.479  19.291  19.691  492362604        0   
       2007-11-13  20.615  21.897  19.691  21.765  484373501        0   
       2007-11-14  22.733  22.739  20.970  21.274  403585172        0   
       2007-11-15  21.280  21.717  20.528  21.040  414487458        0   
       2007-11-16  21.193  21.388  20.405  21.309  385660112        0   

                   Daily Return  7d MA  30d MA  Rolling Volatility (30d)  
Ticker Date                                                               
aapl   2007-11-12           NaN    NaN     NaN                       NaN  
       2007-11-13     10.532731    NaN     NaN                       NaN  
       2007-11-14     -2.255915    NaN     NaN                       NaN  
       2007-11-15     -1.099934    NaN     NaN                       NaN  
       2007-11-16      1.278517    NaN

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [8]:
highest_avg_return_stock = df_filtered.groupby(level="Ticker")["Daily Return"].mean().idxmax()
highest_avg_return_value = df_filtered.groupby(level="Ticker")["Daily Return"].mean().max()

print(f"Stock with highest average return: {highest_avg_return_stock} ({highest_avg_return_value:.2f}%)")

Stock with highest average return: tsla (0.17%)


In [9]:
df_filtered["YearMonth"] = df_filtered.index.get_level_values("Date").to_period("M") 
volatility_per_month = df_filtered.groupby(["Ticker", "YearMonth"])["Daily Return"].std()
most_volatile_ticker, most_volatile_month = volatility_per_month.idxmax()
most_volatile_value = volatility_per_month.max()

print(f"Most volatile month: {most_volatile_month} for {most_volatile_ticker} (Volatility: {most_volatile_value:.2f}%)")

Most volatile month: 2010-06 for tsla (Volatility: 61.16%)
